In [ ]:
import os
import re
import nltk
import numpy as np
import json
import csv
import random
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('stopwords')

In [2]:
TOTAL = True

with open("../pre/global/not_prune.csv", "r") as f:
    not_prune = list(csv.reader(f, delimiter=","))[0]

if TOTAL:
    directory = "../data"

    with open("../pre/global/not_prune.csv", "r") as f:
        data_list = not_prune
else:
    directory = "../pre/sample"

    data_list = os.listdir(directory)
    data_list = [data for data in data_list if data in not_prune]

In [3]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', device='cuda')

In [4]:
doc_embeddings = {}
stop_words = set(stopwords.words('portuguese'))
for file in data_list:
    with open(f'{directory}/{file}', 'r') as f:
        text = f.read()
        text = text.lower()
        text = re.sub(r'[^\w0-9- ]+', '', text, flags=re.UNICODE)
        text = [x.strip() for x in text.split() if len(x) > 0 and x not in stop_words]
        text = ' '.join(text)

        doc_embeddings[file] = list(map(float, (model.encode(text, device='cuda'))))

In [5]:
if TOTAL:
    with open("../pre/global/full_embed.json", 'w') as f: json.dump(doc_embeddings, f)
else:
    with open("../pre/global/embed.json", 'w') as f: json.dump(doc_embeddings, f)

In [6]:
def get_similar_docs(document_file:str, embed_json_path:str="../pre/global/full_embed.json", max_recs:int|str=5, simplified:bool=False) -> list|int:

    if max_recs is not int and max_recs != 'MAX':
        print(f'Invalid value of maximum recommendations! It must  be either \'MAX\' or an interger. Current value = {max_recs}')
        return -1
    
    with open(embed_json_path, 'r') as f:
        embed_dict = json.load(f)

    if simplified:
        doc_embeds = np.array(embed_dict[document_file])
    else: 
        stop_words = set(stopwords.words('portuguese'))
        with open(document_file, 'r') as f:
            text = f.read()
            text = text.lower()
            text = re.sub(r'[^\w0-9- ]+', '', text, flags=re.UNICODE)
            text = [x.strip() for x in text.split() if len(x) > 0 and x not in stop_words]
            text = ' '.join(text)

            doc_embeds = np.array(model.encode(text, device='cuda'))
    
    
    results = []
    for file in embed_dict.keys():
        embed_value = np.array(embed_dict[file])
        similarity = cosine_similarity(doc_embeds.reshape(1, -1), embed_value.reshape(1, -1))
        results.append((file, similarity[0][0]))


    ordered_results = sorted(results, reverse=True, key=lambda x:x[1])

    if max_recs is int and len(ordered_results) >= max_recs+1:
        return ordered_results[1: max_recs+1]

    return ordered_results[1:]

In [ ]:
sample_num = 10
test_docs = random.sample(not_prune, k=sample_num)

max_total = 0
min_total = 0
total = 0
for doc in test_docs:
    print(f'Documento em questão: {doc}')
    results = get_similar_docs(doc, max_recs='MAX', simplified=True)
    print(results[:5])
    average = sum([value[1] for value in results])/len(results)
    max_total += results[0][1]
    min_total += results[-1][1]
    total += average
    print(f'Maximum Similarity = {results[0]}')
    print(f'Minimum Similarity = {results[-1]}')
    print(f'Average Similarity = {average}')
    print()

print()
print(f'Maxima geral de similaridade entre documentos: {max_total/sample_num}')
print(f'Minima geral de similaridade entre documentos: {min_total/sample_num}')
print(f'Média geral de similaridade entre documentos: {total/sample_num}')